### Which running backs have had 50+ carries in each of the last 5 years?

This notebook scrapes data from [Pro Football Reference](https://www.pro-football-reference.com/)'s Rushing and Receiving data table and places it into a data frame ([sample table](https://www.pro-football-reference.com/years/2017/rushing.htm)). The notebook uses the `Requests` and `Beautiful Soup 4` modules to gather the web page data. A `Player` class is used to create objects representing a single player. The program loops through the 5 most recent NFL seasons and gathers data for each season. A column for the player's fantasy points for the season is added to each data frame. The points total is based on a standard Yahoo! league (0 PPR). The scoring can be changed in the `FANTASY_SETTINGS_DICT` dictionary. A data frame for each season is placed in a list, and this list is concatenated into one big data frame. Then, various manipulations are made to the data frame to find all running backs who have had 50 or more rushing attempts in each of the last 5 years. The final data frame is saved as a `.csv` file.

** Note: **This dataset does not include 2 point conversions. This will affect the fantasy point total for some players. Luckily, few running backs get many 2 point conversions (9 RB's in 2017 each had only 1 conversion; the rest had 0)  This will only have a small affect on the point total for some of the running backs.

Import the needed modules:

In [1]:
import requests
import bs4
import pandas as pd
import numpy as np
import datetime

The `Player` class is used to represent a single player and their stats for a given season:

In [2]:
class Player(object):
    """
    The Player class is used to represent a record in the dataset. The player's 'name' and 'year' attributes will be 
    used in the data frame as a multi-hierarchical index. The class uses the HEADER dictionary to assign attributes 
    and use the appropriate datatype.
    """
    def __init__(self, data, HEADER):
        """
        Initialize the Player object. This method will use the keys and values in HEADER to assign attributes and 
        data types for the attributes.
        """
        # Loop through the HEADER dictionary keys and values. An enumeration is also used to grab data from a specific
        # column in the row.
        for i, (attr, data_type) in enumerate(HEADER.items()):
            # Remove unwanted characters in data.
            # '*' in the player's name indicates a Pro Bowl appearance.
            # '+' in the player's name indicates a First-Team All-Pro award.
            # '%' is included in the catch percentage stat on pro-football reference
            if data[i].endswith('*+'):
                data[i] = data[i][:-2]
            elif data[i].endswith('%') or data[i].endswith('*') or data[i].endswith('+'):
                data[i] = data[i][:-1]
                
            # Set the class attribute. If the data is empty then we will assign a NumPy NaN value to the attribute.
            # Otherwise, we set the attribute as usual.
            if not data[i]:
                setattr(self, type(data[i])(np.NaN), str(data[i]))
            else:
                setattr(self, attr, data_type(data[i]))

These functions are used to scrape the data and create the data frame:

In [3]:
def scrape_data(dataframe_list, stat_year, end_year, current_year, HEADER):
    """
    This function goes to pro-football-reference.com 'Rushing and Receiving' page for the 
    given season and scrapes the data from the webpage. A Player object is used to represent 
    the data for each player. The function uses a list called dataframe_list to create a 
    data frame for each year of data and append that data frame to the end of the list. The list 
    is then returned.
    """
    # Send a GET request to Pro Football Reference's Rushing & Receiving page to gather the data.
    r = requests.get('https://www.pro-football-reference.com/years/' + str(current_year) + '/rushing.htm')
    r.raise_for_status()

    # Create a BeautifulSoup object.
    soup = bs4.BeautifulSoup(r.text, 'lxml')

    # Find the first table with tag 'table' and id 'rushing_and_receiving
    table = soup.find('table', id='rushing_and_receiving')

    # tbody is the table's body
    # Get the body of the table
    body = table.find('tbody')

    # tr refers to a table row
    # Each row in player_list has data for a player
    # This will also collect descriptions of each column found in the web page's table.
    # We will filter this out in the for loop below.
    player_list = body.find_all('tr')

    # This list holds a dictionary of each object's attributes.
    # The dictionary is made from the object's __dict__ attribute.
    player_dict_list = []

    # Get each player's stats, create a Player object, and append the object
    # and the instance's __dict__ to their own list
    for player in player_list:
        # The <td> tag defines a standard cell in an HTML table.
        # Get a list of cells. This data represents one player
        info_list = player.find_all('td')
        
        # If info_list has data, then we will extract the desired information from the elements.
        # info_list will be empty if the current 'player' in the player_list is actually other 
        # irrelevant information we're not interested in (such as a column description).
        if info_list:
            # Collect the relevant info from the cells and place them in player_info
            player_info = []
            for stat in info_list:
                player_info.append(stat.text)
                
            # Create a Player object and append the __dict__ attribute to a list.
            # This list is used for the data in our data frame.
            obj = Player(player_info, HEADER)
            player_dict_list.append(obj.__dict__)
            
    return player_dict_list

In [4]:
def make_dataframe(dataframe_list, player_dict_list, year, HEADER):
    """
    Creates a new data frame and inserts the the data frame into dataframe_list. A 'year' and a 
    'fantasy_points' column is added to the data frame. The 'year' column indicates the year of 
    the season, and the 'fantasy_points' column calculates how many fantasy points the player 
    would have learned that season (excluding 2 point conversions).
    """
    
    # This dictionary holds the stat name as keys and their fantasy points worth as values.
    FANTASY_SETTINGS_DICT = {
        'pass_yard': 1/25, # 25 yards = 1 point
        'pass_td': 4,
        'interception': -1,
        'rush_yard': 1/10, # 10 yards = 1 point
        'rush_td': 6,
        'rec_yard': 1/10, # 10 yards = 1 point
        'reception': 0,
        'receiving_td': 6,
        'two_pt_conversion': 2,
        'fumble_lost': -2,
        'offensive_fumble_return_td': 6,
        'return_yard': 1/25 # 25 yards = 1 point
    }
    
    # Create the data frame, add column headers, and add a 'year' column for the current season.
    df = pd.DataFrame(data=player_dict_list)
    header_list = list(HEADER.keys())
    df = df[header_list]
    df['year'] = year
    df['fantasy_points'] = df['rush_yards']*FANTASY_SETTINGS_DICT['rush_yard'] + \
                           df['rush_touchdowns']*FANTASY_SETTINGS_DICT['rush_td'] + \
                           df['rush_touchdowns']*FANTASY_SETTINGS_DICT['rush_td'] + \
                           df['receptions']*FANTASY_SETTINGS_DICT['reception']
    
    # Add the data frame to the data frame list.
    dataframe_list.append(df)

In [5]:
def modify_data(dataframe_list, NUM_YEARS):
    """
    This function takes a list of data frames as input. It concatenates the data frames together 
    to create one large data frame. It then modifies the data to get the running backs with 50 or 
    more carries in each of the past NUM_YEARS season. The modified data frame is returned.
    """
    # Concatenate the data frames to create one large data frame that has data for each season.
    big_df = pd.concat(dataframe_list)

    # The concatenation creates duplicate indexes, so we will reset the index
    big_df.reset_index(inplace=True, drop=True)

    # Eliminate players with fewer than 50 rush attempts.
    big_df = big_df[big_df['rush_attempts'] >= 50]

    # Some players have a NaN value their position. Any player with 50 or more rush attempts 
    # but no position is likely a running back.
    big_df['position'] = big_df['position'].fillna('RB')

    # Some running backs have their position description in lowercase letters.
    # Use a lambda function to fix this inconsistency.
    big_df['position'] = big_df['position'].apply(lambda x: 'RB' if 'rb' in x else x)

    # Only interested in running backs
    big_df = big_df[big_df['position'] == 'RB']

    # Set the player's name and the season's year as the indexes.
    big_df = big_df.set_index(['name', 'year'])

    # Sort the data frame by the player's name
    big_df.sort_index(level='name', inplace=True)

    # Get each player's name
    names = big_df.index.get_level_values('name').unique()

    # Loop through each player. If they don't have NUM_YEARS season's worth of data, then we 
    # drop them from the dataset.
    for name in names:
        if len(big_df.loc[name]) != NUM_YEARS:
            big_df.drop(name, inplace=True)
            
    return big_df

We can now execute the functions and create the final data frame:

In [6]:
# Get the current date to help figure out which year to start gathering data from.
now = datetime.datetime.now()

# Number of years of data.
NUM_YEARS = 5

# Starting with last year since it's a full season of data.
# Regular season football ends in late December or early January, so if the current 
# date is late December, and the season has already ended, then unfortunately the 
# newly created season will not be included in the dataset.
# Football season starts the week after Labor Day weekend. Labor Day is always on 
# the first Monday of September, and the NFL regular season is 17 weeks long 
# (16 games + bye week for a team). From my initial calculations, if September 1 is 
# Labor Day, then the NFL regular season ends on December 28.
start_year = now.year - 1

# Get the final year to gather data from.
end_year = start_year - NUM_YEARS

# This dictionary is used for creating columns in the data frame and assigning a datatype for each column.
HEADER = {
    'name': str,
    'team': str,
    'age': int,
    'position': str,
    'games_played': int,
    'games_started': int,
    'rush_attempts': int,
    'rush_yards': int,
    'rush_touchdowns': int,
    'longest_run': int,
    'yards_per_rush': float,
    'yards_per_game': float,
    'attempts_per_game': float,
    'targets': int,
    'receptions': int,
    'rec_yards': int,
    'yards_per_rec': float,
    'rec_touchdowns': int,
    'longest_rec': int,
    'rec_per_game': float,
    'rec_yards_per_game': float,
    'catch_percentage': float,
    'scrimmage_yards': int,
    'rush_rec_touchdowns': int,
    'fumbles': int
}

# This dictionary holds the stat name as keys and their fantasy points worth as values.
FANTASY_SETTINGS_DICT = {
    'pass_yard': 1/25, # 25 yards = 1 point
    'pass_td': 4,
    'interception': -1,
    'rush_yard': 1/10, # 10 yards = 1 point
    'rush_td': 6,
    'rec_yard': 1/10, # 10 yards = 1 point
    'reception': 0,
    'receiving_td': 6,
    'two_pt_conversion': 2,
    'fumble_lost': -2,
    'offensive_fumble_return_td': 6,
    'return_yard': 1/25 # 25 yards = 1 point
}


# First, we need to scrape the data from Pro-Football Reference.

# Holds each data frame scraped from Pro-Football Reference.
# Each data frame has data for a single season.
dataframe_list = []

# Iterate through each year of data and create a data frame for each one.
for year in range(start_year, end_year, -1):
    player_dict_list = scrape_data(dataframe_list, start_year, end_year, year, HEADER)
    make_dataframe(dataframe_list, player_dict_list, year, HEADER)


# The data has been scraped. We will now concatenate the data frames and clean the data.
big_df = modify_data(dataframe_list, NUM_YEARS)


# Write the data frame to a csv file and save it in the current working directory.
big_df.to_csv('rb_data.csv')

The dimensions of the data frame (number of rows, number of columns):

In [7]:
big_df.shape

(65, 25)

Number of running backs with 50 or more carriers in each of the past 5 seasons (2013-2017):

In [8]:
len(big_df.index.get_level_values('name').unique())

13

The name of each player with 50 or more rush attempts in each of the last 5 seasons:

In [9]:
for name in big_df.index.get_level_values('name').unique():
    print(name)

Alfred Morris
Chris Ivory
DeMarco Murray
Doug Martin
Eddie Lacy
Frank Gore
Giovani Bernard
Lamar Miller
Le'Veon Bell
LeGarrette Blount
LeSean McCoy
Mark Ingram
Matt Forte


The dataset:

In [10]:
big_df

team  age position  games_played  games_started  \
name              year                                                   
Alfred Morris     2013  WAS   25       RB            16             16   
                  2014  WAS   26       RB            16             16   
                  2015  WAS   27       RB            16             16   
                  2016  DAL   28       RB            14              0   
                  2017  DAL   29       RB            14              5   
Chris Ivory       2013  NYJ   25       RB            15              6   
                  2014  NYJ   26       RB            16             10   
                  2015  NYJ   27       RB            15             14   
                  2016  JAX   28       RB            11              1   
                  2017  JAX   29       RB            14              3   
DeMarco Murray    2013  DAL   25       RB            14             14   
                  2014  DAL   26       RB            16             16   
                  2015  PHI   27       RB            15              8   
                  2016  TEN   28       RB            16             16   
                  2017  TEN   29       RB            15             15   
Doug Martin       2013  TAM   24       RB             6              6   
                  2014  TAM   25       RB            11             11   
                  2015  TAM   26       RB            16             16   
                  2016  TAM   27       RB             8              8   
                  2017  TAM   28       RB            11              8   
Eddie Lacy        2013  GNB   22       RB            15             15   
                  2014  GNB   23       RB            16             16   
                  2015  GNB   24       RB            15             12   
                  2016  GNB   25       RB             5              5   
                  2017  SEA   26       RB             9              3   
Frank Gore        2013  SFO   30       RB            16             16   
                  2014  SFO   31       RB            16             16   
                  2015  IND   32       RB            16             16   
                  2016  IND   33       RB            16             16   
                  2017  IND   34       RB            16             16   
...                     ...  ...      ...           ...            ...   
Lamar Miller      2013  MIA   22       RB            16             15   
                  2014  MIA   23       RB            16             16   
                  2015  MIA   24       RB            16             16   
                  2016  HOU   25       RB            14             14   
                  2017  HOU   26       RB            16             13   
Le'Veon Bell      2013  PIT   21       RB            13             13   
                  2014  PIT   22       RB            16             16   
                  2015  PIT   23       RB             6              6   
                  2016  PIT   24       RB            12             12   
                  2017  PIT   25       RB            15             15   
LeGarrette Blount 2013  NWE   27       RB            16              7   
                  2014  2TM   28       RB            16              1   
                  2015  NWE   29       RB            12              6   
                  2016  NWE   30       RB            16              8   
                  2017  PHI   31       RB            16             11   
LeSean McCoy      2013  PHI   25       RB            16             16   
                  2014  PHI   26       RB            16             16   
                  2015  BUF   27       RB            12             12   
                  2016  BUF   28       RB            15             15   
                  2017  BUF   29       RB            16             16   
Mark Ingram       2013  NOR   24       RB            11              3   
                  2014  NOR

No running back has a season with fewer than 50 rush attempts:

In [12]:
big_df['rush_attempts'].min()

69